In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd
from scipy.integrate import simps, trapz, cumtrapz
from scipy.ndimage.filters import gaussian_filter1d
from time import time
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 9]

In [2]:
%run -i "../../carbon_walls/gcmc.py"

In [3]:
filename = "../data/" + "y0.75rep2" + ".csv"
data_dump = pd.read_csv(filename).copy()

for i in range(data_dump.shape[0]):
    if np.isnan( data_dump.yco_sim[i]):
        yco = data_dump.yco_res[i]
        P_res = data_dump.P_res[i]*10**5 #[Pa]
        T = data_dump.T_res[i] + 273.15 #K
        fco, fme = PR_Fugacity( P_res/10**5, T, yco )
        fco = fco*10**5
        fme = fme*10**5
        
        sf = False
        mega_verbose = False

        s_box = 57.15
        N_max = 50000
        Vol = s_box**3
        kb = 1.3806*10**(7) #[Pa*A^3/K]
        Nco = 0 #floor(fco*Vol/kb/T)
        Nme = 0 #floor(fme*Vol/kb/T)
        Nc = 0
        rc = s_box
        beta = 1/T
        zz_co = beta*fco
        zz_me = beta*fme
        delta = 1
        pi_move = 0.5

        N_moves = 100
        N_equil = int( np.round( 200000/N_moves) )
        N_prod = int( np.round( 100000/N_moves) )

        t0 = time()
        rhocov,rhomev,Env,Pv,Ncov, Nmev = mc_run()
        t = time()
        
        Nco = Ncov.mean()
        Nme = Nmev.mean()
        data_dump.yco_sim[i] = Nco/(Nco + Nme)
        data_dump.P_sim[i] = Pv.mean()*10 #[bar]
        data_dump.E_sim[i] = Env.mean()
        data_dump.rhoco[i] = rhocov.mean()
        data_dump.rhome[i] = rhomev.mean()
        data_dump.yco_s[i] = (rhocov/(rhocov + rhomev)).std()
        data_dump.P_s[i] = Pv.std()*10 #[bar]
        data_dump.rhoco_s[i] = rhocov.std()
        data_dump.rhome_s[i] = rhomev.std()
        data_dump.E_s[i] = Env.std()
        data_dump.time[i] = t - t0
        print( i, data_dump.yco_sim[i],  data_dump.P_sim[i], data_dump.rhoco[i], data_dump.rhome[i], data_dump.time[i] )
        #filename = "../data_output/yco" + str(yco) + "P" + str(P) + "/run" + str(i) + ".csv"
        data_dump.to_csv( filename, index = False)

0 0.7481142323276048 10.100857726388934 0.00018721169918064324 6.303310447477209e-05 33.36271142959595
1 0.7435480986688825 19.565906567766888 0.00037334078189321174 0.0001287663212808469 67.74379682540894
2 0.7432926790223183 29.71130866555114 0.0005845592119556619 0.00020188632753835722 98.04548692703247
3 0.7364998223388631 38.37351278598468 0.0007867583560793856 0.00028148135317257206 133.66722989082336
4 0.7421695250480084 48.11748775865179 0.0010400584654144476 0.00036131740669128167 178.92600083351135
5 0.7260391502897534 55.60432942475032 0.0012200696507503209 0.00046037643850451614 223.89210391044617
6 0.7302419892858522 65.37762163694764 0.0015536817619761418 0.0005739441274849413 263.3572287559509
7 0.7232990740018969 72.37125501663564 0.001785092049926601 0.0006828940350686077 309.79850339889526
8 0.7053258477440886 82.58736922068698 0.002049064138956508 0.000856067078778142 362.9304850101471
9 0.7099411323070461 95.23684157096542 0.002639764713550792 0.0010785220477367715 